<img src="image/image.jpg" alt="Tableau Meta" width="300" height="300" style="float:left; margin-right: 40px; margin-bottom: 20px;" />

# Banner to Smartsheet Data Migration  
### Migrating Data from Banner to Smartsheet using SQL and Python  

<div style="clear: both;"></div>  

## Disclaimer  
This content is designed for **beginner-level** team members, so concepts and terminology are explained in a simplified manner.  


---

## **Purpose & Overview**  
This notebook provides a **baseline of fundamental tools and methods** for **extracting, transforming, and loading (ETL) data** between different systems. The goal is to introduce **repeatable, adaptable workflows** that can be **repurposed** for various data migration scenarios, including:  

- Moving structured data from **Banner (or other databases) to Smartsheet**  
- Automating **data extraction, transformation, and validation** using SQL and Python  
- Scheduling **automated updates** to keep destination systems in sync with source data  

By learning these foundational concepts and workflows, you will be able to **extend these techniques** to support **a wide range of ETL use cases** beyond this specific migration process.  

---




---

## **Table of Contents**  
### **Prerequisites**  
Before proceeding, ensure you meet the following requirements:  

- You have access to a [SQL Server database](https://ts.vcu.edu/about-us/computer-center/database-administration/sql-server/) with **Admin rights** or know the Admin who can make updates.  
- A database is set up to store ODS data (a new one can be created if you have Admin privileges).  
- You have **ownership or admin rights** to the target database.  
- **Microsoft SQL Server Management Studio (SSMS)** is installed on your local machine, and you have VPN access.  
- You have a development environment (IDE) set up for **Python 3.9** and Jupyter Notebooks (`pip install -r requirements.txt`).  
- You have an active **Smartsheet license**.  
- **Security & Compliance:** You have **explicit approval** from **InfoSec and the Banner team** to use your Banner data in Smartsheet.  
- **Best Practice:** In production environments, use a **service account** for production ownership roles.  

---

## **Step 1: Overview**  
This step covers the fundamental setup and concepts required before migrating data.  

### **1.1 Review IDE Setup**  
- **Virtual environments (`venv`)**  
- **Notebook operations**  
- **`requirements.txt` usage**  
- **Folder and file structure**  

### **1.2 Review SQL Management Studio (SSMS)**  
- **Tables and views**  
- **SQL operations**  

### **1.3 Smartsheet Setup**  
- **API Key**  
- **Sheet ID**  
- **Field requirements**  

---

## **Step 2: Walkthrough**  
A step-by-step guide to migrating data.  

### **2.1 Install Dependencies**  
- Install the required libraries:  
  ```bash
  pip install -r requirements.txt

### **2.2 Import Required Modules**  
- Import necessary Python libraries for database access and API interaction.  

### **2.3 Set Up Smartsheet**  
- Obtain API token and Sheet ID.

### **2.4 Configure Variables**  
- Update credentials and API keys in your script.

### **2.5 Initialize Objects**  
- Create database connections and Smartsheet API objects.

### **2.6 Load Test Data into SQL & Create a SQL View**  
- Import sample data into SQL Server.
- Create a SQL view for extracting relevant data.

### **2.7 Load SQL Data into Smartsheet & Test API Methods**  
- Extract data from SQL Server and upload it to Smartsheet.

### **2.8 Copy ODS Tables to Your Database**  
- Perform data transfer from the ODS system to your target database.

### **2.9 Create a Stored Procedure to Refresh ODS Data**  
- Automate data updates using SQL stored procedures.

### **2.10 Schedule an Automated Data Refresh**  
- Configure a SQL Agent Job to refresh ODS data on a schedule.

### **2.11 Create an Optional View from ODS Data Tables**  
- Build a SQL view to facilitate easier data access and analysis.

### **2.12 Review & Validate Data**  
- Ensure data is correctly migrated and matches expectations.



## **Step 3: Tips & Tricks** 

### **3.1 Can We Load Data Directly from the ODS Linked Server?**  
- ✅ Yes, but it depends on access permissions and performance considerations.

### **3.2 Other API Data Considerations**  
- Handling different data structures via API calls.

### **3.3 Can Python Jobs be Automated/Scheduled?**  
- ✅ Yes, but it depends machine access and error checking.

---

## **Additional Resources**  

### **Smartsheet API Documentation**  
For more details on Smartsheet API methods, refer to the **[Smartsheet API Reference](https://smartsheet.redoc.ly/)**.  

### **Download SQL Server Management Studio (SSMS)**  
[Download Here](https://learn.microsoft.com/en-us/sql/ssms/download-sql-server-management-studio-ssms?view=sql-server-ver16)  

---

## **IDE Recommendations**  
Choosing an IDE depends on your experience and long-term needs:  

### **Beginner-Friendly (Quick Setup, but Limited Long-Term Scalability)**  
- **[Anaconda](https://www.anaconda.com/download)** – Easier setup but may become restrictive for larger projects.  

### **More Robust & Flexible (Requires Manual Setup, but Scalable for Larger Projects)**  
- **[Visual Studio Code](https://code.visualstudio.com/download)** – A lightweight but powerful IDE for Python development.  


---
---


## **Step 1: Overview**  
This step covers the fundamental setup and concepts required before migrating data.  

### **1.1 Review IDE Setup**  
- **Virtual environments (`venv`)**  
- **Notebook operations**  
- **`requirements.txt` usage**  
- **Folder and file structure**  

### **1.2 Review SQL Management Studio (SSMS)**  
- **Tables and views**  
- **SQL operations**  

### **1.3 Smartsheet Setup**  
- **API Key**  
- **Sheet ID**  
- **Field requirements**  



---
---

## **Step 2: Walkthrough**  
A step-by-step guide to migrating data.  

## Note: The following cell only needs to be ran during the initial implementation
### **2.1 Install Dependencies**  
- Install the required libraries:  
  ```bash
  pip install -r requirements.txt

In [ ]:
#!pip install -r requirements.txt # comment out when done loading

### **2.2 Import Required Modules**  
- Import necessary Python libraries for database access and API interaction.  

In [ ]:
# Run the following cell to import the required packages each time you start a new session 
import pyodbc
import concurrent.futures
import sqlalchemy
from sqlalchemy import Column, BigInteger, DateTime, Integer, String, Enum as EnumColumn, Float, create_engine, text, Table, inspect, MetaData, types 
from sqlalchemy.orm import column_property, sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from functions.utliz_sql import *
from functions.smartsh import _SysSmartsheetServices
import pandas as pd
import smartsheet
import requests
import json

### **2.3 Set Up Smartsheet**  
- Obtain API token and Sheet ID.
    - Create a Smartsheet workspace then create a "Grid" file
    - It's assumed you may not have data in your SQL database so we'll use a dataset about movies for testing  
        - Use movie_date_field_names.csv found in the sample_data folder and import this file to quickly build the field titles. Note: make sure to save the file
    - Get your Sheet ID by going to File->Properties and copy the value found in the field <b>Sheet ID</b> and store for later use
    - Create your Smartsheet API token by following these [API Token Steps](https://help.smartsheet.com/articles/2482389-generate-API-key) and store somewhere safe for later use
    - Add your Sheet ID and API token to the following cell in their respective areas in the following cell within the quoation marks

### **2.4 Configure Variables**  
- Update credentials and API keys in your script.

In [ ]:
server = "REPLACE_TEXT_HERE"
database = 'REPLACE_TEXT_HERE'
username = "REPLACE_TEXT_HERE"
password = "REPLACE_TEXT_HERE" 
smartsheet_api_key = 'REPLACE_TEXT_HERE'
sheet_id = 'REPLACE_TEXT_HERE'

### **2.5 Initialize Objects**  
- Create database connections and Smartsheet API objects.

In [ ]:
# Initialize the smartsheet and sql objects
smartsheet_init = _SysSmartsheetServices(smartsheet_api_key)
_sys_info_services = SysInfoServices(server, database, username, password)

### **2.6 Load Test Data into SQL & Create a SQL View**  
- Import sample data into SQL Server.
- Create a SQL view for extracting relevant data.

In [ ]:
# Because it's assumed you may not have Banner data in your SQL database we'll load a testing dataset about movies into your database
# using two tables that we'll combine into one view prior to loading into Smartsheet. Logic can be updated to use your target Banner data

# load the 2 source csv filed into 2 dataframes 
table1_df = pd.read_csv('sample_data/movie_data_t1.csv') 
table2_df = pd.read_csv('sample_data/movie_data_t2.csv') 

table1_df.head(3)

In [ ]:
# Create/update SQL tables using the dataframes in your target database
_sys_info_services.update_sql_table_with_dataframe(table1_df, 'test_movie_data_t1')
_sys_info_services.update_sql_table_with_dataframe(table2_df, 'test_movie_data_t2')

- create view by copy/pasting the following into the SQL query via the image below
   
        -- Use CREATE for inital build and ALTER when updating
        --ALTER VIEW vw_test_movie_demo AS -- uncomment this to update while commenting out the CREATE line
            CREATE VIEW vw_test_movie_demo AS
            SELECT
                t1.[show_id],
                t1.[title], 
                t1.[director], 
                t1.[cast], 
                t1.[country], 
                t1.[date_added],
                t1.[release_year],
                t2.[type], 
                t2.[rating], 
                t2.[duration],
                t2.[listed_in],
                t2.[description]
            FROM smartsheet.dbo.test_movie_data_t1 t1
            LEFT JOIN smartsheet.dbo.test_movie_data_t2 t2
                ON t1.[show_id] = t2.[show_id];

<img src="image/new_query.png" alt="Create New Job" width="300" height="300" style="margin-left: 10px; margin-bottom: 40px; margin-top: 20px;" /> 





### **2.7 Load SQL Data into Smartsheet & Test API Methods**  
- Extract data from SQL Server and upload it to Smartsheet.


In [ ]:
# Get your view/table into a dataframe
source_df =  _sys_info_services.get_table_or_view_data('vw_test_movie_demo')

In [ ]:
# Limit the number of records for testing
source_df_limited = source_df.iloc[0:10]

# Load your source data into Smartsheet
smartsheet_init.refresh_source_data(source_df_limited, sheet_id)

#### extra code - sample methods

In [ ]:
# load smartsheet data into a dataframe
sm_to_df = smartsheet_init.extract_and_load_to_dataframe_with_row_id(sheet_id)

sm_to_df_no_row_id = sm_to_df.drop(columns=['row_id']) # removes a row id from the result set

sm_to_df_no_row_id.head(3)

In [ ]:
# add new data to the top of the Smartsheet without deleting all the data in the sheet prior to loading
source_df_new_records = source_df.iloc[10:20]

smartsheet_init.add_records_to_top_of_smartsheet(source_df_new_records, sheet_id)

In [ ]:
# delete all data in the Smartsheet
smartsheet_init.delete_sheet_data(sheet_id)


### **2.8 Copy ODS Tables to Your Database**  
- Perform data transfer from the ODS system to your target database.

    -- Copy a table from one db to another using the statements below and replacing text as needed
    - Select * into smartsheet.dbo.testDemo from [ODS_CLOUD]..[ONCORE].[ONCORE_ODS];
    - Select * into smartsheet.dbo.testDemo2 from [ODS_CLOUD]..[ONCORE].[ONCORE_ODS];
    
<img src="image/new_query.png" alt="Create New Job" width="300" height="300" style="margin-left: 10px; margin-bottom: 40px; margin-top: 20px;" />

### **2.9 Create a Stored Procedure to Refresh ODS Data**  
- Automate data updates using SQL stored procedures.

<img src="image/stored_proc_location.png" alt="Create New Job" width="300" height="300" style="margin-left: 10px; margin-bottom: 40px; margin-top: 10px;" />

    -- Use CREATE for inital build and ALTER when updating
    CREATE PROCEDURE SP_REFRESH_ODS_DATA
    --ALTER PROCEDURE SP_REFRESH_ODS_DATA
    AS
    BEGIN
        SET NOCOUNT ON;

        -- Truncate table testDemo
        TRUNCATE TABLE smartsheet.dbo.testDemo;

        -- Insert data into testDemo
        INSERT INTO smartsheet.dbo.testDemo (
            [PROTOCOL_NO], 
            [IRB_NO], 
            [PI], 
            [CLINICAL_TRIAL], 
            [ORGANIZATION_UNIT], 
            [DEPARTMENT_NAME], 
            [MGMT_GROUP_DESCRIPTION], 
            [CURRENT_STATUS], 
            [OPEN_TO_ACCRUAL_DATE], 
            [TOTAL_ENROLLMENT], 
            [TARGET_ACCRUAL_UPPER], 
            [STUDY_SITE_CONTACT], 
            [SPONSOR_NAME], 
            [SPONSOR_TYPE_DESCRIPTION], 
            [FUND_ACCOUNTNO], 
            [GRANT_ACCOUNT], 
            [CLINICAL_TRIAL_FINANCE_STAFF], 
            [PI_V_NUMBER]
        )
        SELECT 
            [PROTOCOL_NO], 
            [IRB_NO], 
            [PI], 
            [CLINICAL_TRIAL], 
            [ORGANIZATION_UNIT], 
            [DEPARTMENT_NAME], 
            [MGMT_GROUP_DESCRIPTION], 
            [CURRENT_STATUS], 
            [OPEN_TO_ACCRUAL_DATE], 
            [TOTAL_ENROLLMENT], 
            [TARGET_ACCRUAL_UPPER], 
            [STUDY_SITE_CONTACT], 
            [SPONSOR_NAME], 
            [SPONSOR_TYPE_DESCRIPTION], 
            [FUND_ACCOUNTNO], 
            [GRANT_ACCOUNT], 
            [CLINICAL_TRIAL_FINANCE_STAFF], 
            [PI_V_NUMBER]
        FROM [ODS_CLOUD]..[ONCORE].[ONCORE_ODS];

        -- Truncate table testDemo2
        TRUNCATE TABLE smartsheet.dbo.testDemo2;

        -- Insert data into testDemo2
        INSERT INTO smartsheet.dbo.testDemo2 (
            [PROTOCOL_NO], 
            [IRB_NO], 
            [PI], 
            [CLINICAL_TRIAL], 
            [ORGANIZATION_UNIT], 
            [DEPARTMENT_NAME], 
            [MGMT_GROUP_DESCRIPTION], 
            [CURRENT_STATUS], 
            [OPEN_TO_ACCRUAL_DATE], 
            [TOTAL_ENROLLMENT], 
            [TARGET_ACCRUAL_UPPER], 
            [STUDY_SITE_CONTACT], 
            [SPONSOR_NAME], 
            [SPONSOR_TYPE_DESCRIPTION], 
            [FUND_ACCOUNTNO], 
            [GRANT_ACCOUNT], 
            [CLINICAL_TRIAL_FINANCE_STAFF], 
            [PI_V_NUMBER]
        )
        SELECT 
            [PROTOCOL_NO], 
            [IRB_NO], 
            [PI], 
            [CLINICAL_TRIAL], 
            [ORGANIZATION_UNIT], 
            [DEPARTMENT_NAME], 
            [MGMT_GROUP_DESCRIPTION], 
            [CURRENT_STATUS], 
            [OPEN_TO_ACCRUAL_DATE], 
            [TOTAL_ENROLLMENT], 
            [TARGET_ACCRUAL_UPPER], 
            [STUDY_SITE_CONTACT], 
            [SPONSOR_NAME], 
            [SPONSOR_TYPE_DESCRIPTION], 
            [FUND_ACCOUNTNO], 
            [GRANT_ACCOUNT], 
            [CLINICAL_TRIAL_FINANCE_STAFF], 
            [PI_V_NUMBER]
        FROM [ODS_CLOUD]..[ONCORE].[ONCORE_ODS];
        END;



### **2.10 Schedule an Automated Data Refresh**  
- Configure a SQL Agent Job to refresh ODS data on a schedule.  

    - 2.10.01 Create new job  

      <img src="image/schedule_new_job.png" alt="Create New Job" width="300" height="300" style="margin-left: 10px; margin-bottom: 40px; margin-top: 20px;" />

    - 2.10.02 Name new job and provide a description along with making sure you're the owner  

      <img src="image/name_job.png" alt="Name New Job" width="700" height="400" style="margin-left: 10px; margin-bottom: 40px; margin-top: 20px;" />

    - 2.10.03 Create a new "Step" with a clear title and update the statement below with your stored proc and add this to the "Command" section  

      <img src="image/create_new_step.png" alt="Name New Job" width="500" height="500" style="margin-left: 10px; margin-bottom: 40px; margin-top: 20px;" />
      <img src="image/new_step_name.png" alt="Name New Job" width="700" height="500" style="margin-left: 10px; margin-bottom: 40px; margin-top: 20px;" />

    - 2.10.04 Go to the "Advanced" tab and using the "On success action" select "Quit the job reporting success" and select OK  

      <img src="image/quit_the_job.png" alt="Name New Job" width="900" height="600" style="margin-left: 10px; margin-bottom: 40px; margin-top: 20px;" />

    - 2.10.05 Use the "Schedules" tab to schedule as needed  

      <img src="image/schedule_job_frequency.png" alt="Name New Job" width="700" height="400" style="margin-left: 10px; margin-bottom: 40px; margin-top: 20px;" />

    - 2.10.06 Use the "Notifications" tab to email when a job fails  

      <img src="image/job_notifications.png" alt="Name New Job" width="800" height="500" style="margin-left: 10px; margin-bottom: 40px; margin-top: 20px;" />

    - 2.10.07 Select OK when done  






### **2.11 Create an Optional View from ODS Data Tables**  
- Build a SQL view to facilitate easier data access and analysis.
   
        -- Use CREATE for inital build and ALTER when updating
        --  ALTER VIEW vw_testDemo AS
            CREATE VIEW vw_testDemo AS
            SELECT 
                t1.[PROTOCOL_NO], 
                t1.[IRB_NO], 
                t1.[PI], 
                t1.[CLINICAL_TRIAL], 
                t2.[ORGANIZATION_UNIT], 
                t2.[DEPARTMENT_NAME], 
                t2.[MGMT_GROUP_DESCRIPTION], 
                t2.[CURRENT_STATUS]
            FROM smartsheet.dbo.testDemo t1
            LEFT JOIN smartsheet.dbo.testDemo2 t2
                ON t1.[IRB_NO] = t2.[IRB_NO];

<img src="image/new_query.png" alt="Create New Job" width="300" height="300" style="margin-left: 10px; margin-bottom: 40px; margin-top: 20px;" /> 



### **2.12 Review & Validate Data**  
- Ensure data is correctly migrated and matches expectations.

---
---

## **Step 3: Tips & Tricks** 

### **3.1 Can We Load Data Directly from the ODS Linked Server?**  
- ✅ Yes, but it depends on access permissions and performance considerations.

In [ ]:
# Fetch from the linked server
# Case 1: When the Linked Server Does Not Require a Database Name
linked_df = _sys_info_services.get_table_or_view_data_from_linked_sql_server(
    linked_server="ODS_CLOUD",
    linked_db=None,  # ✅ Uses ".." format
    schema="VCU_CUSTOM", 
    table_name="VCU_SUMMARY_RESIDENCY"
)


linked_df.head(3)

In [ ]:
# Fetch from the linked server
# Case 2: When the Linked Server Requires a Database Name
linked_df = _sys_info_services.get_table_or_view_data_from_linked_sql_server(
    linked_server="ODS_CLOUD",
    linked_db="VCU_CUSTOM",  # ✅ Database is required
    schema="dbo", 
    table_name="VCU_SUMMARY_RESIDENCY"
)

linked_df.head(3)

### **3.2 Other API Data Considerations**  
- Handling different data structures via API calls.

In [ ]:
# https://open-meteo.com/en/docs
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"hourly": "temperature_2m"
}

response = requests.get(url, params)
response.json()['hourly']

In [ ]:
weather_df = pd.DataFrame(response.json()['hourly'])
weather_df

### **3.3 Can Python Jobs be Automated/Scheduled?**  
- ✅ Yes, but it depends machine access and error checking.

In [ ]:
import schedule
import threading
import time
from datetime import datetime as dt


print('scheduler started')




# create function to process via the schedule
def sql_to_smartsheet_etl():
    '''
    funcation to move the data from SQL to smartsheet
    '''
    source_df =  _sys_info_services.get_table_or_view_data('vw_test_movie_demo')

    # Limit the number of records for testing
    source_df_limited = source_df.iloc[0:10]

    # Load your source data into Smartsheet
    smartsheet_init.refresh_source_data(source_df_limited, sheet_id)




# run jobs in sepearte threads to avoid race conditions or other timing issues that would affect jobs starting
def run_threaded(job_func):
    job_thread = threading.Thread(target=job_func)
    job_thread.start()




# build the jira_etl scheduler
def smartsheet_etl_scheduler():
    '''
    scope:
    funtion that helps schedule the extraction of data from SQL to land in smartsheet

    '''


    # this is the listing of M-F as provided via datetime
    weekdays = [0, 1, 2, 3, 4]

    # get today at as int to check the list above to see if today is a weekday
    today_as_int = dt.now().weekday()

    ####### the following logic is used to determine if updates should occur and then processes accordingly

    # The following logic checks conditions to determine if the following funtion should run
    if today_as_int in weekdays:
        print('start data extraction')
        # funtion to extract data from polygon
        sql_to_smartsheet_etl()

        print('etl finished')

    # if not a weekday:
    else:
        pass

smartsheet_etl_scheduled = schedule.Scheduler()






# schedule the jobs for 3am each M-F
smartsheet_etl_scheduled.every().day.at('03:00').do(run_threaded, smartsheet_etl_scheduler) # make sure your refresh schedule is sync with the SQL refresh to allow lag time 




# loop the jobs forever until the process fails or system restarts 
while True:

    smartsheet_etl_scheduled.run_pending()


    time.sleep(1)


# END